In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Problem 1
## Dataset Generation

Write a function to **generate a training set** of size $m$
- randomly generate a weight vector $w \in \mathbb{R}^{10}$, normalize length
- generate a training set $\{(x_i , y_i)\}$ of size m
  - $x_i$: random vector in $\mathbb{R}^{10}$ from $\textbf{N}(0, I)$
  - $y_i$: $\{0, +1\}$ with $P[y = +1] = \sigma(w \cdot x_i)$ and $P[y = 0] = 1 - \sigma(w \cdot x_i)$

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def generate_data(m):
    # returns the true w as well as X, Y data

    w = np.random.rand(10)
    norm = np.linalg.norm(w)
    global normalized_w
    normalized_w = w / norm
    training_set = np.zeros((m, 11))
    
    for i in range(m):
        x = np.random.normal(0, 1, 10)
        sigma = sigmoid(np.dot(normalized_w, x))
        y_range = [0, 1]
        y_prob = [1-sigma, sigma]
        y = np.random.choice(a = y_range, p = y_prob)
        training_set[i] = np.append(x,y)
    return training_set

## Algorithm 1: logistic regression

The goal is to learn $w$.  Algorithm 1 is logistic
  regression (you may use the built-in method LogisticRegression for this. Use max_iter=1000).

In [16]:
from sklearn.linear_model import LogisticRegression

def logistic_regression_function(m_array):
    average_difference_norm = np.zeros(len(m_array))
    for idx, num in enumerate(m_array):
        difference_norm = np.zeros(10)
        for i in range(10):
            training_set = generate_data(num)
            X = training_set[:, :-1]
            y = training_set[:, -1]
            lr = LogisticRegression(max_iter=1000)
            lr.fit(X, y)
            global normalized_w
            norm = np.linalg.norm(normalized_w - lr.coef_)
            difference_norm[i]= norm
        average_difference_norm[idx] = difference_norm.mean()
    return average_difference_norm

## Algorithm 2: gradient descent with square loss

Define square loss as
$$L_i(w^{(t)}) = \frac{1}{2} \left( \sigma(w^{(t)} \cdot x) - y_i \right)^2$$

  Algorithm 2 is
  gradient descent with respect to square loss (code this
  up yourself -- run for 1000 iterations, use step size eta = 0.01).

In [22]:
def gradient_descent_with_square_loss(m_array):
    average_difference_norm = np.zeros(len(m_array))
    for idx, num in enumerate(m_array):
        difference_norm = np.zeros(10)
        for i in range(10):
            training_set = generate_data(num)
            X = training_set[:, :-1]
            y = training_set[:, -1]
            
            w_backtick = np.random.randn(10)
            step_size_eta = 0.01
            iterations = 1000
            for j in range(iterations):
                predicted_y = sigmoid(np.dot(X, w_backtick))
                gradient = np.dot(X.T, predicted_y - y)
                gradient /= 10
                gradient *= step_size_eta
                w_backtick -= gradient 
            global normalized_w
            norm = np.linalg.norm(normalized_w - w_backtick)
            difference_norm[i]= norm
        average_difference_norm[idx] = difference_norm.mean()
    return average_difference_norm
m_array = [50, 100, 150, 200, 250]
#m_array = [2]
gradient_descent_with_square_loss(m_array)

array([1.34035234, 0.87155809, 0.62210739, 0.59616378, 0.46144609])

## Algorithm 3: stochastic gradient descent with square loss
Similar to gradient descent, except we use the gradient at a single random training point every iteration.

## Evaluation

Measure error $\|w - \hat{w}\|_2$ for each method at different sample size. For any
  fixed value of $m$, choose many different $w$'s and average the
  values $\|w - 
  \hat{w}\|_2$ for Algorithms 1, 2 and 3.  Plot the results
  for for each algorithm as you make $m$ large (use $m=50, 100, 150, 200, 250$).
  Also record, for each algorithm, the time taken to run the overall experiment.

In [9]:
import time

m_array = [50, 100, 150, 200, 250]
start = time.process_time()
print(logistic_regression_function(m_array))
time_taken = time.process_time() - start

[1.08655887 0.78456794 0.74136165 0.52659305 0.39129404]


# Problem 2

In [10]:
from sklearn import datasets

In [11]:
cancer = datasets.load_breast_cancer()

For each depth in $1, \dots, 5$, instantiate an AdaBoost classifier with the base learner set to be a decision tree of that depth (set `n_estimators=10` and `learning_rate=1`), and then record the 10-fold cross-validated error on the entire breast cancer data set. Plot the resulting curve of accuracy against base classifier depth. Use $101$ as your random state for both the base learner as well as the AdaBoost classifier every time.